In [1]:
import tensorflow as tf;
import numpy as np;
import pandas as pd;
import matplotlib.pyplot as plt;
import sklearn;

In [2]:
def unfold(X,k):
    n = X.shape[0];
    p1 = X.shape[1];
    p2 = X.shape[2];
    if k == 0 :
        X = np.transpose(X,(1,2,0));
        Xk = np.reshape(X,[p1,n*p2],order='F')
    else :
        X = np.transpose(X,(2,1,0));
        Xk = np.reshape(X,[p2,p1*n],order='F')
    return Xk ;

In [3]:
def plot(fig,u,v,i1,i2):
    image = u @ v.T;
    ax = fig.add_subplot(5,5,i1*5+i2+1)
    ax.set_title("a{} b{}^T".format(i1+1,i2+1))
    img = ax.imshow(image,cmap='gray',interpolation='nearest',origin='upper')
    cbar = fig.colorbar(img,ax=ax,aspect=50,pad=0.08,shrink=0.5,orientation='horizontal')

# Preprocessing

In [4]:
imgX = 224;
imgY = 224;

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

prefix = 'chest_xray';

train_generator = train_datagen.flow_from_directory(
   prefix + '/train',
   target_size=(imgX,imgY),
   batch_size=5216,
   class_mode="input",
   color_mode="grayscale",
   shuffle=False,
)

validation_generator = validation_datagen.flow_from_directory(
    prefix + '/val',
    target_size=(imgX,imgY),
    class_mode="input",
    color_mode="grayscale",
    shuffle=False,
    batch_size=16,
)

test_generator = test_datagen.flow_from_directory(
    prefix+'/test',
    target_size=(imgX,imgY),
    class_mode="input",
    color_mode="grayscale",
    shuffle=False,
    batch_size=624,
)

num_classes = 2;

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
X,Y = train_generator.next();

n = X.shape[0];
p1 = X.shape[1];
p2 = X.shape[2]; 

X = np.reshape(X,[n,p1,p2]);

In [6]:
test,test_ = test_generator.next();

m = test.shape[0];
p1 = test.shape[1];
p2 = test.shape[2]; 

test = np.reshape(test,[m,p1,p2]);
test1 = np.transpose(test,(1,2,0));
test1 = np.reshape(test1,[p1*p2,m],order='F');

# MPCA

In [7]:
X1 = np.transpose(X,(1,2,0));
X1 = np.reshape(X1,[p1, p2*n],order='F');
U1,S1,V1 = np.linalg.svd(X1 @ X1.T);

X2 = np.transpose(X,(2,1,0));
X2 = np.reshape(X2,[p2, p1*n],order='F');
U2,S2,V2 = np.linalg.svd(X2 @ X2.T);

In [8]:
A = U1[:,0:32]; # 3,10,20,32,50
B = U2[:,0:32]; # 3,10,20,32,50

In [9]:
i = 1;
count = 5;

for X,Y in train_generator:
    n = X.shape[0];
    p1 = X.shape[1];
    p2 = X.shape[2];
    X = np.reshape(X,[n,p1,p2]);
    
    Bn = np.expand_dims(B,0)
    Bn = np.tile(Bn,[n,1,1]);
    XB = np.matmul(X,Bn);
    XB2 = unfold(XB,0);
    U1,S1,V1 = np.linalg.svd(XB2 @ XB2.T)
    A = U1[:,0:32]; # 3,10,20,32,50
    
    ATn = np.expand_dims(A.T,0)
    ATn = np.tile(ATn,[n,1,1]);
    AX = np.matmul(ATn,X);
    AX1 = unfold(AX,1);
    U2,S2,V2 = np.linalg.svd(AX1 @ AX1.T)
    B = U2[:,0:32]; # 3,10,20,32,50
    
    AATn = np.expand_dims(A @ A.T,0);
    AATn = np.tile(AATn,[n,1,1]);
    BBTn = np.expand_dims(B @ B.T,0);
    BBTn = np.tile(BBTn,[n,1,1]);
    X_ = AATn @ X @ BBTn;
    error = np.sum(np.sum(np.square(X-X_),axis=1),axis=1);
    print(np.mean(error))
    
    i = i+1;
    if i > count:
       break;

80.5898
80.58907
80.58907
80.58907
80.58907


# Derive MPCA-reduced core tensor

In [10]:
An = np.expand_dims(A, 0);
An = np.tile(An, [n,1,1]);
AnT = np.transpose(An, (0,2,1));
Bn = np.expand_dims(B,0);
Bn = np.tile(Bn, [n,1,1]);
Xc = AnT @ X @ Bn;

Am = np.expand_dims(A, 0);
Am = np.tile(Am, [m,1,1]);
AmT = np.transpose(Am, (0,2,1));
Bm = np.expand_dims(B,0);
Bm = np.tile(Bm, [m,1,1]);
test_c = AmT @ test @ Bm;

In [11]:
Xsvm = np.reshape(Xc,[n,32*32]); # 3,10,20,32,50
testsvm = np.reshape(test_c,[m,32*32]); # 3,10,20,32,50

In [12]:
Ysvm = np.array([0,1])
Ysvm = np.repeat(Ysvm,[1341,3875],axis=0)
Tsvm = np.array([0,1])
Tsvm = np.repeat(Tsvm,[234,390],axis=0)

In [13]:
from sklearn.svm import SVC;

### Linear SVM

In [14]:
svm1 = SVC(kernel='linear',probability=False);
svm1.fit(Xsvm,Ysvm)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [15]:
error1 = 0
for i, v in enumerate(svm1.predict(testsvm)):
    if v!= Tsvm[i]:
        error1+=1
print('Accuracy :',round(1 - error1/624,4))

Accuracy : 0.7436


### Nonlinear SVM with Gaussian kernel

In [16]:
svm2 = SVC(kernel='rbf',probability=False,gamma='auto');
svm2.fit(Xsvm,Ysvm)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [17]:
error2 = 0
for i, v in enumerate(svm2.predict(testsvm)):
    if v!= Tsvm[i]:
        error2+=1
print('Accuracy :',round(1 - error2/624,4))

Accuracy : 0.7724


### Logistic regression using Single Neuron Neural Network

In [100]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.BatchNormalization(axis=-1,momentum=0.999999,epsilon=0.001,center=True,scale=True,beta_initializer='zeros',
                                             gamma_initializer='ones',moving_mean_initializer='zeros',moving_variance_initializer='ones'));
model.add(tf.keras.layers.Dense(1,activation='sigmoid',input_shape=(32*32,))) # 3,10,20,32,50
model.compile(loss='binary_crossentropy',optimizer='RMSprop',metrics=['accuracy'])
history = model.fit(Xsvm,Ysvm,batch_size=250,epochs=20,verbose=1,validation_data=(testsvm,Tsvm))

Train on 5216 samples, validate on 624 samples
Epoch 1/20
5216/5216 [==============================] - 1s 172us/sample - loss: 0.5847 - accuracy: 0.7099 - val_loss: 0.5518 - val_accuracy: 0.7099
Epoch 2/20
5216/5216 [==============================] - 0s 15us/sample - loss: 0.3245 - accuracy: 0.8756 - val_loss: 0.4911 - val_accuracy: 0.7740
Epoch 3/20
5216/5216 [==============================] - 0s 15us/sample - loss: 0.2349 - accuracy: 0.9156 - val_loss: 0.5134 - val_accuracy: 0.7869
Epoch 4/20
5216/5216 [==============================] - 0s 14us/sample - loss: 0.1892 - accuracy: 0.9319 - val_loss: 0.5721 - val_accuracy: 0.7708
Epoch 5/20
5216/5216 [==============================] - 0s 11us/sample - loss: 0.1589 - accuracy: 0.9457 - val_loss: 0.6366 - val_accuracy: 0.7628
Epoch 6/20
5216/5216 [==============================] - 0s 16us/sample - loss: 0.1394 - accuracy: 0.9525 - val_loss: 0.6578 - val_accuracy: 0.7724
Epoch 7/20
5216/5216 [==============================] - 0s 14us/sample